In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests

In [3]:
import pdfkit

In [4]:
# Import data

output_relevant_history = pd.read_csv("C:/Users/Bajo/OneDrive/Desktop/GMU GRA Project/Antidepressants_database/output_Relevant_History.csv")
subgroups_and_optimal_antidepressants = pd.read_csv("C:/Users/Bajo/OneDrive/Desktop/GMU GRA Project/Antidepressants_database/Subgroups_and_Optimal_Antidepressants.csv")

In [5]:
# Join the tables

strata_join = pd.merge(output_relevant_history, subgroups_and_optimal_antidepressants, how='left', left_on='strata_sort', right_on='strata_sort')

In [6]:
# Identify relevant columns

strata_join = strata_join[['strata_id', 'antidep', 'a+b', 'prob', 'Age', 'Gender', 'strata_sort_1', 'strata_sort_2', 'strata_sort_3', 'strata_sort_4', 'strata_sort_5', 'strata_sort_6', 'strata_sort_7', 'strata_sort_8', 'strata_sort_9']]

In [7]:
# Add a flag for when there are not enough matched cases

strata_join['a+b_is_-1_flag'] = strata_join['a+b'].apply(lambda x: 1 if x == '-1' else 0)

In [8]:
# Add a flag for when the only information provided is the gender and age

only_gender_id_list = [12377, 13679, 13772, 3817, 18388, 7200, 17162, 16458, 7704, 211]
strata_join['only_gender_id_flag'] = strata_join['strata_id'].isin(only_gender_id_list).astype(int)

In [9]:
# Add a flag for when the remission rate is less than 10%

strata_join['less_than_10_percent_prob_flag'] = (strata_join['prob'] < 0.1).astype(int)

In [10]:
# Columns to capitalize

columns_to_capitalize = ['antidep', 'strata_sort_1', 'strata_sort_2', 'strata_sort_3',
                     'strata_sort_4', 'strata_sort_5', 'strata_sort_6', 'strata_sort_7',
                     'strata_sort_8', 'strata_sort_9']

In [11]:
# Capitalize each word in a string

def capitalize_each_word(value):
    if isinstance(value, str):
        # Splitting the string into words, capitalizing each, and joining back together
        return ' '.join(word.capitalize() for word in value.split())
    return value

# Apply the function to each specified column
for column in columns_to_capitalize:
    strata_join[column] = strata_join[column].apply(capitalize_each_word)

In [12]:
# Clean strata_sort columns

# List of columns to remove parentheses and their contents
columns_to_modify = ['strata_sort_1', 'strata_sort_2', 'strata_sort_3',
                     'strata_sort_4', 'strata_sort_5', 'strata_sort_6', 
                     'strata_sort_7', 'strata_sort_8', 'strata_sort_9']

for column in columns_to_modify:
    # Explicitly convert column to string to ensure .str accessor works correctly.
    # This conversion will turn NaN/None values into the string 'nan', which we can handle in the regex if necessary
    strata_join[column] = strata_join[column].astype(str)

    # Now remove parentheses and their contents, including the case where the cell might have 'nan'
    # The regular expression '\s*\([^)]*\)|^nan$' matches any character sequence starting with '(' and ending with ')'
    # including the parentheses themselves, optionally preceding whitespace, or matches exactly 'nan'
    strata_join[column] = strata_join[column].str.replace('\s*\([^)]*\)|^nan$', '', regex=True)

# After this operation, if you want to convert 'nan' strings back to NaN values, you can do another replacement
strata_join.replace('nan', pd.NA, inplace=True)

In [14]:
# Replace "Last Ad: Same & No Remiss" in strata_sort columns

columns_to_update = ['strata_sort_1', 'strata_sort_2', 'strata_sort_3', 'strata_sort_4', 'strata_sort_5', 'strata_sort_6', 'strata_sort_7', 'strata_sort_8', 'strata_sort_9']

for column in columns_to_update:
    strata_join[column] = strata_join[column].replace('Last Ad: Same & No Remiss', 'Used the same antidepressant within last year with no remission.')

for column in columns_to_update:
    strata_join[column] = strata_join[column].replace('Last Ad: Diff & No Remiss', 'Used a different antidepressant within last year with no remission.')

for column in columns_to_update:
    strata_join[column] = strata_join[column].replace('Last Ad: Diff & Remiss', 'Used a different antidepressant within last year with remission.')

In [15]:
# Create Relevant History text

def insert_relevant_history(row):
    if row['only_gender_id_flag'] == 1:
        return f"<p>According to our data, you are {row['Age']} years old and of the {row['Gender']} Gender. None of the 16,770 patient profiles in our database correspond to your specific situation. As a result, we are unable to provide data-driven guidance tailored to you. Please rely on the judgment of your clinician regarding what is best for you.</p>"
    else:
        # Dynamically create a list of non-empty and non-NaN strata_sort items
        strata_sort_items = [f"<li>{row[f'strata_sort_{i}']}</li>" for i in range(1, 10) if pd.notna(row[f'strata_sort_{i}']) and row[f'strata_sort_{i}'] != '']
        
        # Join the list items into a single string
        strata_sort_html = "".join(strata_sort_items)
        
        return f"""<p>According to our data, you are {row['Age']} years old, of the {row['Gender']} Gender, and have a history of taking the following medications/have a history of the following relevant illnesses:</p><ul>{strata_sort_html}</ul><p>If you believe an important aspect of your medical history has been overlooked, please mention it to your clinician when you provide them with this report.</p>"""

# Applying the function to each row in the DataFrame and creating the new column
strata_join['relevant_history'] = strata_join.apply(insert_relevant_history, axis=1)

In [16]:
# Create a flag for if the recommendations url exists

def recommendation_url_exists(row):
    session = requests.Session()
    url = f"https://hi.gmu.edu/ad_files/bar_results/bar_graph/barstrataID{row['strata_id']}.png"
    try:
        response = session.head(url, timeout=5)
        return 1 if response.status_code == 200 else 0
    except requests.RequestException:
        return 0

# Assuming 'strata_join' is your DataFrame and it has been defined correctly
strata_join['recommendation_url_exists'] = strata_join.apply(recommendation_url_exists, axis=1)

In [17]:
# Create the Recommendations graphs file

def determine_recommendations_graphs_value(row):
    url = f"https://hi.gmu.edu/ad_files/bar_results/bar_graph/barstrataID{row['strata_id']}.png"
    
    if row['a+b_is_-1_flag'] == 0 and row['only_gender_id_flag'] == 0 and row['less_than_10_percent_prob_flag'] == 0 and row['recommendation_url_exists'] == 1:
        return f'<figure><img src="{url}" width="600" height="400"></figure>'
    else:
        return None

# Apply the function to create the new column
strata_join['recommendations_graphs'] = strata_join.apply(determine_recommendations_graphs_value, axis=1)

In [18]:
# Create a flag for if the executive summary url exists

def executive_summary_url_exists(row):
    session = requests.Session()
    url = f"https://hi.gmu.edu/ad_files/Piechart_9/StrataID{row['strata_id']}.png"
    try:
        response = requests.head(url, timeout=5)
        return 1 if response.status_code == 200 else 0
    except requests.RequestException:
        return 0

# Apply the function to the DataFrame
strata_join['executive_summary_url_exists'] = strata_join.apply(executive_summary_url_exists, axis=1)

In [19]:
# Create the Executive Summary graphs file

def determine_executive_summary_graphs_value(row):
    url = f"https://hi.gmu.edu/ad_files/Piechart_9/StrataID{row['strata_id']}.png"
    if row['a+b_is_-1_flag'] == 0 and row['only_gender_id_flag'] == 0 and row['less_than_10_percent_prob_flag'] == 0 and row['executive_summary_url_exists'] == 1:
        return f'<figure><img src="{url}" alt="{row["antidep"]} is Best for You" width="500" height="300"><figcaption>Figure 1: {row["antidep"]} is Best for You</figcaption></figure>'
    else:
        return None

# Apply the function to create the new column
strata_join['executive_summary_graphs'] = strata_join.apply(determine_executive_summary_graphs_value, axis=1)

In [ ]:
recommendations_graphs = strata_join['recommendations_graphs']

In [ ]:
# Mapping of antidepressants to their NIH links

nih_links = {
    'Amitriptyline': 'https://www.ncbi.nlm.nih.gov/books/NBK537225/',
    'Bupropion': 'https://www.ncbi.nlm.nih.gov/books/NBK470212/',
    'Citalopram': 'https://www.ncbi.nlm.nih.gov/books/NBK482222/',
    'Desvenlafaxine': 'https://www.ncbi.nlm.nih.gov/books/NBK534829/',
    'Doxepin': 'https://www.ncbi.nlm.nih.gov/books/NBK542306/',
    'Duloxetine': 'https://www.ncbi.nlm.nih.gov/books/NBK549806/',
    'Escitalopram': 'https://www.ncbi.nlm.nih.gov/books/NBK557734/',
    'Fluoxetine': 'https://www.ncbi.nlm.nih.gov/books/NBK459223/',
    'Mirtazapine': 'https://www.ncbi.nlm.nih.gov/books/NBK519059/',
    'Nortriptyline': 'https://www.ncbi.nlm.nih.gov/books/NBK482214/',
    'Paroxetine': 'https://www.ncbi.nlm.nih.gov/books/NBK526022/',
    'Ropinirole': 'https://www.ncbi.nlm.nih.gov/books/NBK554532/',
    'Sertraline': 'https://www.ncbi.nlm.nih.gov/books/NBK547689/',
    'Trazodone': 'https://www.ncbi.nlm.nih.gov/books/NBK470560/',
    'Venlafaxine': 'https://www.ncbi.nlm.nih.gov/books/NBK548799/',
    'Other': 'https://www.ncbi.nlm.nih.gov/books/NBK538182/'
}
# Adding a new column 'antidep_link' to the table based on 'antidep' column values
strata_join['antidep_link'] = strata_join['antidep'].map(nih_links)

In [ ]:
# Convert probability to a percentage

strata_join['prob_percent'] = (strata_join['prob'] * 100).astype(int)

In [ ]:
# Create Executive Summary text

def generate_executive_summary(row):
    if row['only_gender_id_flag'] == 1:
        return '<p> In the absence of any matched medical history, we advise you to consult a licensed healthcare professional to explore suitable treatment options.</p>'
    elif row['less_than_10_percent_prob_flag'] == 1:
        return '<p>There were not enough cases that matched your medical history, so we advise you to consult a licensed healthcare professional to explore suitable treatment options.</p>'
    elif row['a+b_is_-1_flag'] == 0 and row['only_gender_id_flag'] == 0 and row['less_than_10_percent_prob_flag'] == 0:
        return f'''<p1> Among patients with similar medical history as you, {row['antidep']} gives you the highest chances for remission. Only {row['prob_percent']}% of patients who took it experienced remission.</p1><br> {row['executive_summary_graphs']} <p3> Out of the 15 common antidepressants we examined, none had remission rates that exceeded {row['prob_percent']}%</p3><p>Learn more about Medications: <a href="{row['antidep_link']}">{row['antidep']}</a></p>'''
    else:
        return '<p>There are no specific oral antidepressants recommended for you.</p>'

# Assuming the DataFrame is named stata_join and has the necessary columns
strata_join['executive_summary'] = strata_join.apply(generate_executive_summary, axis=1)

In [ ]:
# Remove duplicate rows

strata_join = strata_join.drop_duplicates()

In [ ]:
# Column for all antidepressants not recommended list

# List of all antidepressants
all_antidepressants = [
    "Amitriptyline", "Bupropion", "Citalopram", "Desvenlafaxine", 
    "Doxepin", "Duloxetine", "Escitalopram", "Fluoxetine", 
    "Mirtazapine", "Nortriptyline", "Paroxetine", "Ropinirole", 
    "Sertraline", "Trazodone", "Venlafaxine", "Other"
]

# Step 1 & 2: Aggregate unique antidepressants per `strata_id` and determine not prescribed
# Note: This assumes the 'antidep' column for each row lists a single antidepressant or 'Other'
grouped_antideps = strata_join.groupby('strata_id')['antidep'].unique()

# Generating the 'not_prescribed' column
def get_not_prescribed(row):
    prescribed = set(grouped_antideps[row['strata_id']])
    not_prescribed = set(all_antidepressants) - prescribed
    not_prescribed_list = list(not_prescribed)
    # Format the list into a string with 'and' if needed
    if len(not_prescribed_list) > 1:
        return ', '.join(not_prescribed_list[:-1]) + ' and ' + not_prescribed_list[-1]
    elif not_prescribed_list:
        return not_prescribed_list[0]
    else:
        return ''

strata_join['not_prescribed'] = strata_join.apply(get_not_prescribed, axis=1)

In [ ]:
# Create Recommendations text

def generate_recommendations(group):
    # Sort by 'recommended_antidep_flag' in descending order, then by 'prob_percent' in descending order
    group = group.sort_values(by=['recommended_antidep_flag', 'prob_percent'], ascending=[False, False])
    
    recommendations = []
    
    for idx, row in group.iterrows():
        # Condition for 'a+b'
        if row['a+b'] == '-1':
            a_b_message = "Few"
        else:
            a_b_message = f"{row['a+b']} persons"
        
        # Condition for 'antidep'
        if row['antidep'] == "Other":
            antidep_message = "uncommon or multiple antidepressants"
        else:
            antidep_message = row['antidep']
        
        # Base recommendation message
        message = f"{a_b_message} tried {antidep_message} and {row['prob_percent']}% experienced remission"
        
        # Append the note for the antidepressant with the highest remission rate
        if idx == group.first_valid_index():
            message += ". This was the antidepressant with the highest remission rate."
        
        # Wrap the message in an <li> tag
        recommendations.append(f"<li>{message}</li>")
    
    # Join all the <li> items into an <ol>
    recommendations_html = "<ol>" + "".join(recommendations) + "</ol>"
    return recommendations_html

# Apply the function and create a mapping from strata_id to recommendations
recommendations_map = strata_join.groupby('strata_id').apply(generate_recommendations)

# Map the recommendations back to the original DataFrame
strata_join['recommendations_percent_list'] = strata_join['strata_id'].map(recommendations_map)

In [ ]:
#Create a recommended_antidep_flag for the actual recommended antidepressant

# Assuming strata_join is your DataFrame
strata_join['recommended_antidep_flag'] = strata_join.sort_values('prob', ascending=False) \
    .groupby('strata_id') \
    .cumcount() == 0

# Convert True/False to 1/0
strata_join['recommended_antidep_flag'] = strata_join['recommended_antidep_flag'].astype(int)

In [ ]:
# Clean the 'a+b' column by removing commas and converting to integer

strata_join['a+b'] = strata_join['a+b'].str.replace(',', '').astype(int)

In [ ]:
# Create a column with the total matched cases

# Filter rows where 'a+b_is_-1_flag' equals 0
filtered_data = strata_join[strata_join['a+b_is_-1_flag'] == 0]

# Group by 'strata_id' and sum 'a+b' within each group
grouped_sum = filtered_data.groupby('strata_id', as_index=False)['a+b'].sum().rename(columns={'a+b': 'matched_cases'})

# Merge this sum back into the original dataframe
# Ensure we are adding the 'matched_cases' as a new column or updating it if it already exists
strata_join = pd.merge(strata_join.drop(columns='matched_cases', errors='ignore'), grouped_sum, on='strata_id', how='left')

# Fill NaNs with 0 in 'matched_cases' and convert to integer
strata_join['matched_cases'] = strata_join['matched_cases'].fillna(0).astype(int)

In [ ]:
# Create Recommendations text

def generate_recommendations(row):
    if row['only_gender_id_flag'] == 1:
        return '<p>Our recommendations are based on the experiences of more than 3 million people with more than 10 million antidepressants. We divide patients into 16,770 profiles, each profile is a unique combination of factors that affect remission. Your features did not match any of our profiles. This suggests that you have a combination of factors that are rare for your age and gender; and for which we do not have sufficient data to provide any advice. It is best to discuss treatment with a clinical provider about which medications may be optimal for you.</p>'
    elif row['less_than_10_percent_prob_flag'] == 1:
        return f'''<p>Less than 10% of {row['matched_cases']} cases that match you responded to any of the 15 common oral antidepressants. These experiences suggest that you may not fully benefit from common oral antidepressants. We recommend that you look at other methods of managing depression besides or in addition to oral antidepressants. These alternatives include medications such as ketamine or esketamine, as well as interventional therapies like transcranial magnetic stimulation (TMS). Each option comes with its own set of considerations regarding efficacy, administration, and potential side effects.</p><p>In addition to exploring alternative treatments in place of oral antidepressants, it is also worth considering augmenting oral antidepressants with your current psychotropic medications. This strategy involves adding another medication to an existing antidepressant regimen, potentially enhancing the therapeutic effects. Augmentation strategies can be an effective path for individuals who do not fully respond to monotherapy and are worth discussing with your healthcare provider.</p><b>How Did We Arrive At Our Conclusions?</b><p>Our recommendations are based on the experiences of more than 3 million people with more than 10 million antidepressants. We divide cases in our data into 16,770 profiles, each profile represents a unique medical history. We have matched your medical history to one of these profiles, that includes {row['matched_cases']} cases similar to you.</p><b>More Information</b><p>For more information about alternative therapies please read the following peer-reviewed published articles:</p><ul><li><a href="https://doi.org/10.30773%2Fpi.2019.0236">Ketamine in Major Depressive Disorder</a></li><li><a href="https://doi.org/10.1007%2Fs44192-022-00012-3">Alternative Treatments for Depression</a></li><li><a href="https://doi.org/10.4103%2Fipj.ipj_88_18">Transcranial magnetic stimulation</a></li><li><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2958866/">Augmentation and Combination Strategies for Treatment-Resistant Depression</a></li></ul><p>We strongly recommend consulting with a licensed professional before considering any changes to your treatment plan. Mental health treatment is highly individualized, and a healthcare provider can offer guidance tailored to your specific situation.</p><p><u>Note:</u> Should you wish to access this report again, kindly note it is designated as Report #{row['strata_id']}.</p>'''
    elif row['a+b_is_-1_flag'] == 0 and row['only_gender_id_flag'] == 0 and row['less_than_10_percent_prob_flag'] == 0 and row['recommended_antidep_flag'] == 1:
        return f'''<p>Remission means that your symptoms will be reduced by 50%. We examined the experiences of more than 3 million patients and more than 10 million antidepressant treatments. Most of these patients did not match your age, gender, and key aspects of your medical history. Among {row['matched_cases']} who did match you, the following remission rates were observed (listed in order of effectiveness): </p>{row['recommendations_percent_list']} {row['recommendations_graphs']} <p>We recommend you take {row['antidep']}.</p> <p>The following antidepressants were not prescribed to patients with your medical background: {row['not_prescribed']}</p><b>How Did We Arrive to Our Conclusions?</b><p>Our recommendations are based on the experiences of more than 3 million people with more than 10 million antidepressants. We divide cases in our data into 16,770 profiles, each profile represents a unique medical history. We have matched your medical history to one of these profiles, that includes {row['matched_cases']} cases similar to you.</p><b>More Information</b><p>For more information on the effectiveness of common antidepressants, see <a href="https://pubmed.ncbi.nlm.nih.gov/34877511/">Effectiveness of common antidepressants: a post market release study.</a></p><p>We strongly recommend consulting with a licensed professional before considering any changes to your treatment plan. Mental health treatment is highly individualized, and a healthcare provider can offer guidance tailored to your specific situation.</p><p><u>Note:</u> Should you wish to access this report again, kindly note it is designated as Report #{row['strata_id']}.</p>'''
    else:
        return "No recommendation available due to missing or incomplete data."

# Apply the function to each row and create a new column for the recommendations
strata_join['Recommendations'] = strata_join.apply(generate_recommendations, axis=1)

In [ ]:
# Create a new table with only values where recommended_antidep_flag = 1

strata_final = strata_join[strata_join['recommended_antidep_flag'] == 1]

In [ ]:
# List of columns to check for missing values

columns_to_check = ['strata_sort_1', 'strata_sort_2', 'strata_sort_3', 'strata_sort_4', 'strata_sort_5', 'strata_sort_6', 'strata_sort_7', 'strata_sort_8', 'strata_sort_9']

for column in columns_to_check:
    # Replace any non-NA/NaN values that may represent missing data with NaN using .loc
    strata_final.loc[strata_final[column] == '', column] = np.nan
    strata_final.loc[strata_final[column] == ' ', column] = np.nan

In [ ]:
# Case 1: Find matches

# Use .copy() to ensure you're working with a DataFrame that won't raise warnings
strata_final = strata_final.copy()

# Adding a column to store HTML outputs for each row based on matches
strata_final['html_output_case_1'] = None

# Optimizing the loop by avoiding iterrows() which is slow for large DataFrames
for index, row in strata_final[pd.isnull(strata_final['strata_sort_2'])].iterrows():
    matches = strata_final[
        (strata_final['Age'] == row['Age']) &
        (strata_final['Gender'] == row['Gender']) &
        (strata_final['strata_sort_1'] == row['strata_sort_1']) &
        (strata_final['strata_sort_2'] != row['strata_sort_2']) &
        (strata_final['strata_sort_2'].notna()) &  # Ensure strata_sort_2 is not null
        (strata_final['strata_sort_3'].isna()) &  # Ensure strata_sort_3 is null
        (strata_final['strata_id'] != row['strata_id'])  # Avoid matching the same row
    ]

    # Formatting matches to HTML
    html_output = '<ul>'
    for _, match in matches.iterrows():
        html_output += f"<li>{match['strata_sort_2']}: We instead recommend <a href='{match['antidep_link']}'>{match['antidep']}</a>.</li>"
    html_output += '</ul>'
    
    # Only update the row if there are matches
    if not matches.empty:
        strata_final.at[index, 'html_output_case_1'] = html_output

In [ ]:
# Case 2: Find matches

# Use .copy() to ensure you're working with a DataFrame that won't raise warnings
strata_final = strata_final.copy()

# Adding a column to store HTML outputs for each row based on matches
strata_final['html_output_case_2'] = None

# Optimizing the loop by avoiding iterrows() which is slow for large DataFrames
for index, row in strata_final[pd.isnull(strata_final['strata_sort_3'])].iterrows():
    matches = strata_final[
        (strata_final['Age'] == row['Age']) &
        (strata_final['Gender'] == row['Gender']) &
        (strata_final['strata_sort_1'] == row['strata_sort_1']) &
        (strata_final['strata_sort_2'] == row['strata_sort_2']) &
        (strata_final['strata_sort_3'] != row['strata_sort_3']) &
        (strata_final['strata_sort_2'].notna()) &  # Ensure strata_sort_2 is not null
        (strata_final['strata_sort_3'].notna()) &  # Ensure strata_sort_3 is not null
        (strata_final['strata_sort_4'].isna()) &  # Ensure strata_sort_4 is null
        (strata_final['strata_id'] != row['strata_id'])  # Avoid matching the same row
    ]

    # Formatting matches to HTML
    html_output = '<ul>'
    for _, match in matches.iterrows():
        html_output += f"<li>{match['strata_sort_3']}: We instead recommend <a href='{match['antidep_link']}'>{match['antidep']}</a>.</li>"
    html_output += '</ul>'
    
    # Only update the row if there are matches
    if not matches.empty:
        strata_final.at[index, 'html_output_case_2'] = html_output

In [ ]:
# Case 3: Find matches

# Use .copy() to ensure you're working with a DataFrame that won't raise warnings
strata_final = strata_final.copy()

# Adding a column to store HTML outputs for each row based on matches
strata_final['html_output_case_3'] = None

# Optimizing the loop by avoiding iterrows() which is slow for large DataFrames
for index, row in strata_final[pd.isnull(strata_final['strata_sort_4'])].iterrows():
    matches = strata_final[
        (strata_final['Age'] == row['Age']) &
        (strata_final['Gender'] == row['Gender']) &
        (strata_final['strata_sort_1'] == row['strata_sort_1']) &
        (strata_final['strata_sort_2'] == row['strata_sort_2']) &
        (strata_final['strata_sort_3'] == row['strata_sort_3']) &
        (strata_final['strata_sort_4'] != row['strata_sort_4']) &
        (strata_final['strata_sort_2'].notna()) &  # Ensure strata_sort_2 is not null
        (strata_final['strata_sort_3'].notna()) &  # Ensure strata_sort_3 is not null
        (strata_final['strata_sort_4'].notna()) &  # Ensure strata_sort_4 is not null
        (strata_final['strata_sort_5'].isna()) &  # Ensure strata_sort_5 is null
        (strata_final['strata_id'] != row['strata_id'])  # Avoid matching the same row
    ]

    # Formatting matches to HTML
    html_output = '<ul>'
    for _, match in matches.iterrows():
        html_output += f"<li>{match['strata_sort_4']}: We instead recommend <a href='{match['antidep_link']}'>{match['antidep']}</a>.</li>"
    html_output += '</ul>'
    
    # Only update the row if there are matches
    if not matches.empty:
        strata_final.at[index, 'html_output_case_3'] = html_output

In [ ]:
# Case 4: Find matches

# Use .copy() to ensure you're working with a DataFrame that won't raise warnings
strata_final = strata_final.copy()

# Adding a column to store HTML outputs for each row based on matches
strata_final['html_output_case_4'] = None

# Optimizing the loop by avoiding iterrows() which is slow for large DataFrames
for index, row in strata_final[pd.isnull(strata_final['strata_sort_5'])].iterrows():
    matches = strata_final[
        (strata_final['Age'] == row['Age']) &
        (strata_final['Gender'] == row['Gender']) &
        (strata_final['strata_sort_1'] == row['strata_sort_1']) &
        (strata_final['strata_sort_2'] == row['strata_sort_2']) &
        (strata_final['strata_sort_3'] == row['strata_sort_3']) &
        (strata_final['strata_sort_4'] == row['strata_sort_4']) &
        (strata_final['strata_sort_5'] != row['strata_sort_5']) &
        (strata_final['strata_sort_2'].notna()) &  # Ensure strata_sort_2 is not null
        (strata_final['strata_sort_3'].notna()) &  # Ensure strata_sort_3 is not null
        (strata_final['strata_sort_4'].notna()) &  # Ensure strata_sort_4 is not null
        (strata_final['strata_sort_5'].notna()) &  # Ensure strata_sort_5 is not null
        (strata_final['strata_sort_6'].isna()) &  # Ensure strata_sort_6 is null
        (strata_final['strata_id'] != row['strata_id'])  # Avoid matching the same row
    ]

    # Formatting matches to HTML
    html_output = '<ul>'
    for _, match in matches.iterrows():
        html_output += f"<li>{match['strata_sort_5']}: We instead recommend <a href='{match['antidep_link']}'>{match['antidep']}</a>.</li>"
    html_output += '</ul>'
    
    # Only update the row if there are matches
    if not matches.empty:
        strata_final.at[index, 'html_output_case_4'] = html_output

In [ ]:
# Case 5: Find matches

# Use .copy() to ensure you're working with a DataFrame that won't raise warnings
strata_final = strata_final.copy()

# Adding a column to store HTML outputs for each row based on matches
strata_final['html_output_case_5'] = None

# Optimizing the loop by avoiding iterrows() which is slow for large DataFrames
for index, row in strata_final[pd.isnull(strata_final['strata_sort_6'])].iterrows():
    matches = strata_final[
        (strata_final['Age'] == row['Age']) &
        (strata_final['Gender'] == row['Gender']) &
        (strata_final['strata_sort_1'] == row['strata_sort_1']) &
        (strata_final['strata_sort_2'] == row['strata_sort_2']) &
        (strata_final['strata_sort_3'] == row['strata_sort_3']) &
        (strata_final['strata_sort_4'] == row['strata_sort_4']) &
        (strata_final['strata_sort_5'] == row['strata_sort_5']) &
        (strata_final['strata_sort_6'] != row['strata_sort_6']) &
        (strata_final['strata_sort_2'].notna()) &  # Ensure strata_sort_2 is not null
        (strata_final['strata_sort_3'].notna()) &  # Ensure strata_sort_3 is not null
        (strata_final['strata_sort_4'].notna()) &  # Ensure strata_sort_4 is not null
        (strata_final['strata_sort_5'].notna()) &  # Ensure strata_sort_5 is not null
        (strata_final['strata_sort_6'].notna()) &  # Ensure strata_sort_6 is not null
        (strata_final['strata_sort_7'].isna()) &  # Ensure strata_sort_7 is null
        (strata_final['strata_id'] != row['strata_id'])  # Avoid matching the same row
    ]

    # Formatting matches to HTML
    html_output = '<ul>'
    for _, match in matches.iterrows():
        html_output += f"<li>{match['strata_sort_6']}: We instead recommend <a href='{match['antidep_link']}'>{match['antidep']}</a>.</li>"
    html_output += '</ul>'
    
    # Only update the row if there are matches
    if not matches.empty:
        strata_final.at[index, 'html_output_case_5'] = html_output

In [ ]:
# Case 6: Find matches

# Use .copy() to ensure you're working with a DataFrame that won't raise warnings
strata_final = strata_final.copy()

# Adding a column to store HTML outputs for each row based on matches
strata_final['html_output_case_6'] = None

# Optimizing the loop by avoiding iterrows() which is slow for large DataFrames
for index, row in strata_final[pd.isnull(strata_final['strata_sort_7'])].iterrows():
    matches = strata_final[
        (strata_final['Age'] == row['Age']) &
        (strata_final['Gender'] == row['Gender']) &
        (strata_final['strata_sort_1'] == row['strata_sort_1']) &
        (strata_final['strata_sort_2'] == row['strata_sort_2']) &
        (strata_final['strata_sort_3'] == row['strata_sort_3']) &
        (strata_final['strata_sort_4'] == row['strata_sort_4']) &
        (strata_final['strata_sort_5'] == row['strata_sort_5']) &
        (strata_final['strata_sort_6'] == row['strata_sort_6']) &
        (strata_final['strata_sort_7'] != row['strata_sort_7']) &
        (strata_final['strata_sort_2'].notna()) &  # Ensure strata_sort_2 is not null
        (strata_final['strata_sort_3'].notna()) &  # Ensure strata_sort_3 is not null
        (strata_final['strata_sort_4'].notna()) &  # Ensure strata_sort_4 is not null
        (strata_final['strata_sort_5'].notna()) &  # Ensure strata_sort_5 is not null
        (strata_final['strata_sort_6'].notna()) &  # Ensure strata_sort_6 is not null
        (strata_final['strata_sort_7'].notna()) &  # Ensure strata_sort_7 is not null
        (strata_final['strata_sort_8'].isna()) &  # Ensure strata_sort_8 is null
        (strata_final['strata_id'] != row['strata_id'])  # Avoid matching the same row
    ]

    # Formatting matches to HTML
    html_output = '<ul>'
    for _, match in matches.iterrows():
        html_output += f"<li>{match['strata_sort_7']}: We instead recommend <a href='{match['antidep_link']}'>{match['antidep']}</a>.</li>"
    html_output += '</ul>'
    
    # Only update the row if there are matches
    if not matches.empty:
        strata_final.at[index, 'html_output_case_6'] = html_output

In [ ]:
# Merge the html_output_case columns 

strata_final['html_output_cases'] = strata_final.apply(lambda row: 
    (str(row['html_output_case_1']) if row['html_output_case_1'] is not None else '') +
    (str(row['html_output_case_2']) if row['html_output_case_2'] is not None else '') +
    (str(row['html_output_case_3']) if row['html_output_case_3'] is not None else '') +
    (str(row['html_output_case_4']) if row['html_output_case_4'] is not None else '') +
    (str(row['html_output_case_5']) if row['html_output_case_5'] is not None else '') +
    (str(row['html_output_case_6']) if row['html_output_case_6'] is not None else ''), axis=1)

In [ ]:
strata_final['html_output_cases'] = strata_final['html_output_cases'].replace(['', ' '], None)

In [ ]:
# Create Exceptions text

def generate_exceptions(row):
    # Checks if flags are 0 and `html_output_cases` is neither None, NaN, nor an empty string
    if (row['a+b_is_-1_flag'] == 0 and row['only_gender_id_flag'] == 0 and row['less_than_10_percent_prob_flag'] == 0) and pd.notnull(row['html_output_cases']) and row['html_output_cases'].strip() != '':
        return f'<p>Considering your specific medical history, as well as potential additional factors, our tailored antidepressant recommendations are designed with precision to enhance your treatment outcomes. Should you experience additional medications, illnesses, and procedures such as:</p> {row["html_output_cases"]} <p>Our approach is to provide a personalized and nuanced treatment plan, aimed at achieving the best possible outcomes for your health and well-being.</p>'
    else:
        return '<p>Given the specifics of your case, consulting a licensed healthcare professional for a suitable treatment option is advised.</p><p>For more information on the effectiveness of common antidepressants, see <a href="https://pubmed.ncbi.nlm.nih.gov/34877511/">Effectiveness of common antidepressants: a post market release study.</a></p>'

# Assuming strata_final is your DataFrame
# Apply the function to each row and create a new column for the exceptions
strata_final['Exceptions'] = strata_final.apply(generate_exceptions, axis=1)

In [ ]:
#Display Data

strata_final

In [88]:
#Export Data

strata_final.to_csv('C:\\Users\\Bajo\\OneDrive\\Desktop\\GMU GRA Project\\Antidepressants_database\\strata_final_table.csv', index=False) 

In [179]:
# Create html Reports column

def create_report(row):
    return f"""<h1><strong>Antidepressant Recommendation Report ID #{row['strata_id']}</strong></h1><h3>Relevant History</h3>{row['relevant_history']}<h3>Executive Summary</h3>{row['executive_summary']}<h3>Recommendations</h3>{row['Recommendations']}<h3>Exceptions</h3>{row['Exceptions']}"""

strata_final['report'] = strata_final.apply(create_report, axis=1)

In [182]:
# Export a sample report

# Filter the DataFrame to find the row with strata_id = 10
row = strata_final[strata_final['strata_id'] == 10]  # Make sure strata_id is treated as the correct type

# Check if the row exists
if not row.empty:
    # Extract the 'report' content
    report_content = row['report'].values[0]
    
    # Define the file path (use an r before the string to avoid issues with backslashes)
    file_path = r'C:\Users\Bajo\OneDrive\Desktop\GMU GRA Project\report_10.html'
    
    # Write the content to an HTML file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(report_content)
    print("Report exported successfully.")
else:
    print("No row found with strata_id = 10.")

Report exported successfully.


In [ ]:
# Export Relevant History's as html's

for index, row in strata_final.iterrows():
    # Generate the file name using the strata_id
    file_name = f"strata_{row['strata_id']}.html"
    
    # Define the file path - modify the base_path as needed to fit your directory structure
    base_path = r"C:\Users\Bajo\OneDrive\Desktop\GMU GRA Project\Relevant History"
    file_path = f"{base_path}\\{file_name}"
    
    # Write the 'report' content to an HTML file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(row['relevant_history'])

In [ ]:
# Export Executive summary's as html's

for index, row in strata_final.iterrows():
    # Generate the file name using the strata_id
    file_name = f"strata_{row['strata_id']}.html"
    
    # Define the file path - modify the base_path as needed to fit your directory structure
    base_path = r"C:\Users\Bajo\OneDrive\Desktop\GMU GRA Project\Executive Summary"
    file_path = f"{base_path}\\{file_name}"
    
    # Write the 'report' content to an HTML file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(row['executive_summary'])

In [ ]:
# Export Recommendations' as html's

for index, row in strata_final.iterrows():
    # Generate the file name using the strata_id
    file_name = f"strata_{row['strata_id']}.html"
    
    # Define the file path - modify the base_path as needed to fit your directory structure
    base_path = r"C:\Users\Bajo\OneDrive\Desktop\GMU GRA Project\Recommendations"
    file_path = f"{base_path}\\{file_name}"
    
    # Write the 'report' content to an HTML file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(row['Recommendations'])

In [ ]:
# Export Exceptions's as html's

for index, row in strata_final.iterrows():
    # Generate the file name using the strata_id
    file_name = f"strata_{row['strata_id']}.html"
    
    # Define the file path - modify the base_path as needed to fit your directory structure
    base_path = r"C:\Users\Bajo\OneDrive\Desktop\GMU GRA Project\Exceptions"
    file_path = f"{base_path}\\{file_name}"
    
    # Write the 'report' content to an HTML file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(row['Exceptions'])

In [185]:
# Export html reports

for index, row in strata_final.iterrows():
    # Generate the file name using the strata_id
    file_name = f"strata_{row['strata_id']}.html"
    
    # Define the file path - modify the base_path as needed to fit your directory structure
    base_path = r"C:\Users\Bajo\OneDrive\Desktop\GMU GRA Project\HTML Reports"
    file_path = f"{base_path}\\{file_name}"
    
    # Write the 'report' content to an HTML file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(row['report'])

In [184]:
# Export pdf reports

for index, row in strata_final.iterrows():
    # Generate the file name using the strata_id
    file_name = f"strata_{row['strata_id']}.pdf"
    
    # Define the file path - modify the base_path as needed to fit your directory structure
    base_path = r"C:\Users\Bajo\OneDrive\Desktop\GMU GRA Project\PDF Reports"
    file_path = f"{base_path}\\{file_name}"
    
    # Write the 'report' content to an HTML file
    pdfkit.from_string(row['report'], file_path)
        file.write(row['report'])